In [252]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [3]:
pd.set_option('display.max_rows',None)

# PoetryDB

In [ ]:
with requests.session() as r:
    authors = r.get(r'https://poetrydb.org/author').json()['authors']
    poems = {}
    json = r.get(fr'https://poetrydb.org/author/{authors[0]}').json()

    poems = {'author': [],
        'title': [],
        'lines': [],
        'linecount' : []}
    

    for author in tqdm(authors):
        json = r.get(fr'https://poetrydb.org/author/{author}').json()
        for poem in json:
            poems['author'].append(poem['author'])
            poems['title'].append(poem['title'])
            poems['lines'].append(poem['lines'])
            poems['linecount'].append(poem['linecount'])

    poems_df = pd.DataFrame(poems)

poems_df


In [ ]:
poems_df.to_csv('poems.csv', mode='w', header = True, index=False)

In [ ]:
poems_df.author.to_csv('poems-authors.csv', header=True, index=False)

# Web Scrape

## Get Poets

In [9]:
with requests.session() as r:
    r.proxies['http'] = 'socks5h://localhost:9050'
    r.proxies['https'] = 'socks5h://localhost:9050'

    
    url = f'https://en.wikipedia.org/wiki/List_of_English-language_poets'
        
    wiki = r.get(url)
    soup = BeautifulSoup(wiki.text, 'html.parser')
    

        

In [10]:
poets_raw = pd.DataFrame()

for group in soup.find_all('div', class_= 'div-col'):
        p_worker = []
        temp_df = pd.DataFrame()
        
        for li in group.find_all('li'):
            p_worker.append(li.text)
        temp_df['poet'] = p_worker
        
        poets_raw = pd.concat([temp_df,poets_raw], axis=0)
       

In [12]:
poets_raw.shape

(3198, 1)

### Write poets_raw DataFrame to csv

In [13]:
poets_raw.to_csv('poets-raw.csv',mode='w',header=True, index=False)

### Reload poets_raw as poets and complete clean

In [14]:
poets = pd.read_csv('poets-raw.csv')

## Find poet nationalities

In [15]:
nationalities = {'A':'australia', 'B': 'barbados', 'Bo': 'bosnia', 'C': 'canada', 'Ch': 'chile', 'Cu':'cuba','D':'dominica','De':'denmark','E':'england','F':'france','G':'germany','Ga':'gambia','Gd':'granada','Gh':'ghana','Gr':'greece'
,'Gu':'guiana','Gy':'guernsey','HK':'hong_kong','In':'india','IoM':'isle_of_man','Is':'israel','Ir':'ireland','It':'italy','J':'jamaica','Je':'jersey','Jp':'japan','K':'kenya','L':'lebanon','M':'malta','Me':'mexico','Mo':'montserrat'
, 'Ne':'nepal','Nf':'newfoundland','Ni':'nigeria','NI':'northern_ireland','Nt':'netherlands','NZ':'new_zealand','P':'pakistan','Pa':'palestine','Ph':'phillipines','PI':'pitcairn_islands','RE':'russian_empire','S':'scotland','SA':'south_africa'
,'Se':'serbia','SL':'saint_lucia','SLe':'sierra_leone','SLk':'sri_lanka','So':'somalia','Sw':'sweden','T':'trinidad_and_tobago','US':'united_states','W':'wales','Z':'zimbabwe'}

In [16]:
poets['nation_key'] = poets.poet.apply(lambda x: x.replace('(','').replace(')','').replace(',','').split()[-1])
poets.nation_key.unique()

array(['US', 'C', 'E', 'A', 'SA', 'US/C', 'Ir', 'S', 'J/C', 'SL', 'E/IoM',
       'E/W', 'W/E', 'PI', 'W', 'C/US', 'Ir/S', 'S/E', 'E/A', 'SLa',
       'Ir/E', 'E/F', 'Ir/NI', 'E/Ir', 'HK', 'J', 'Ir/Z', 'NZ/A', '1653',
       'Ni', 'Ph', 'J/E', 'Se/C', 'In', 'E/US', 'E/C', 'Me/C', 'So/C',
       'C/E', 'A/E', 'NZ', 'C/Ch', 'India', 'SM', 'S/C', 'Bo/C', 'NI',
       'J/US', 'C/De', 'T', '1945', 'Gh', 'US/E', 'Je', 'Ch/C', 'Ir/C',
       'E/Nt', 'Sw', 'C/A', 'NI/E', 'Cu', 'E/S', 'Ga', 'T/C', 'SA/E',
       'Ne', 'W/F', 'S/SA', 'SLk/C', 'Ni/US', 'RE/US', 'NI/Ir', 'Gu/E',
       'A/S', 'W/US', 'SA/C', 'US/SA', 'Mo/E', 'Z/SA', 'Ir]', 'US/De',
       'K/Z', 'Ir/US', 'Ir/SLk', 'Z', 'US/Nt', 'SA/A', 'SA/US', 'SA/Is',
       'Gd/C', 'W/A', 'T/E', 'Ir/I', 'Is/C', 'Nf', 'G/US', 'Gy', 'S/A',
       'L/US', 'US/Jp', 'In/E', 'SA/Nt', 'Gu/C', 'Gu', 'S/US', 'C]',
       'P/W', 'In/US', 'US/A', 'Ir/A', 'T/S', 'SLe', 'Ir/F', 'E/G',
       'US/Ir', 'US/F', 'S/NZ', 'G/C', 'It', '1633', 'B', 'SA/F', 'IoM',


In [17]:
poets.reset_index(drop=True,inplace=True)
poets

,poet,nation_key
0,"Matthew Zapruder (born 1967, US)",US
1,"Marya Zaturenska (1902–1982, US)",US
2,"Robert Zend (1929–1985, C)",C
3,"Benjamin Zephaniah (born 1958, E)",E
4,"Komninos Zervos (born 1950, A)",A
...,...,...
3193,"Pam Ayres (born 1947, E)",E
3194,"Sir Robert Aytoun (1570–1638, S/E)",S/E
3195,"William Edmondstoune Aytoun (1813–1865, S)",S
3196,"Nnorom Azuonye (born 1967, Ni)",Ni


In [18]:
def check_nationality(poet):
    poet = poet.replace('(','').replace(')','').replace(',','')
    nation_key = poet.split()[-1]
    try:
        try:
            return nationalities[nation_key]
        except:
            nation_keys = nation_key.split('/')
            nations = []
            for key in nation_keys:
                nations.append(nationalities[key])
            return nations
    except:
        return np.nan

check_nationality('Matthew Zapruder (born 1967, US)')

'united_states'

In [19]:
check_nationality('J. Michael Yates (1938–2021, US/C)')

['united_states', 'canada']

In [20]:
poets['nationality'] = poets.poet.apply(check_nationality).copy()

poets['nationality_1'] = poets['nationality'].apply(lambda x: x[0] if type(x) == list else x)
poets['nationality_2'] = poets['nationality'].apply(lambda x: x[1] if type(x) == list else np.nan)

In [21]:
poets

,poet,nation_key,nationality,nationality_1,nationality_2
0,"Matthew Zapruder (born 1967, US)",US,united_states,united_states,NaN
1,"Marya Zaturenska (1902–1982, US)",US,united_states,united_states,NaN
2,"Robert Zend (1929–1985, C)",C,canada,canada,NaN
3,"Benjamin Zephaniah (born 1958, E)",E,england,england,NaN
4,"Komninos Zervos (born 1950, A)",A,australia,australia,NaN
...,...,...,...,...,...
3193,"Pam Ayres (born 1947, E)",E,england,england,NaN
3194,"Sir Robert Aytoun (1570–1638, S/E)",S/E,"[scotland, england]",scotland,england
3195,"William Edmondstoune Aytoun (1813–1865, S)",S,scotland,scotland,NaN
3196,"Nnorom Azuonye (born 1967, Ni)",Ni,nigeria,nigeria,NaN


There are 267 poets with dual nationality - I will drop these rows to allow only one column for nationality without null values.

In [22]:
poets.drop(poets[~poets.nationality_2.isnull()].index, inplace=True)
poets.drop('nationality_2', axis=1,inplace=True)
poets.reset_index(drop=True,inplace=True)


In [23]:
poets.drop(['nationality','nation_key'],axis=1,inplace=True)

In [24]:
poets.rename(columns={'nationality_1':'nationality'},inplace=True)

In [25]:
poets['nationality'].value_counts()

united_states          787
england                671
canada                 512
australia              351
scotland               208
ireland                126
south_africa            80
wales                   52
new_zealand             51
jamaica                 15
northern_ireland        14
nigeria                 12
india                   11
trinidad_and_tobago      8
ghana                    3
jersey                   2
guiana                   2
hong_kong                1
newfoundland             1
isle_of_man              1
barbados                 1
italy                    1
sierra_leone             1
guernsey                 1
gambia                   1
zimbabwe                 1
nepal                    1
cuba                     1
sweden                   1
saint_lucia              1
pitcairn_islands         1
phillipines              1
malta                    1
Name: nationality, dtype: int64

## Find poet dates

In [30]:
def find_dates(poet):
    try:
        try:
            pattern = re.compile(r'living')
            match = re.search(pattern, poet)
            return match[0]
        except:
            pattern = re.compile(r'\d{4}(\S\d{4})?')
            match = re.search(pattern, poet)
            return match[0]
    except: 
        return np.nan

In [31]:
poets['dates'] = poets.poet.apply(find_dates).copy()

In [33]:
poets.dates.isnull().sum()

11

#### Drop Null Values

In [35]:
poets.dropna(how='any',subset=['dates'],inplace=True)
poets.reset_index(drop=True, inplace=True)

poets

,poet,nationality,dates
0,"Matthew Zapruder (born 1967, US)",united_states,1967
1,"Marya Zaturenska (1902–1982, US)",united_states,1902–1982
2,"Robert Zend (1929–1985, C)",canada,1929–1985
3,"Benjamin Zephaniah (born 1958, E)",england,1958
4,"Komninos Zervos (born 1950, A)",australia,1950
...,...,...,...
2915,"Ayo Ayoola-Amale (born 1970, Ni)",nigeria,1970
2916,"Pam Ayres (born 1947, E)",england,1947
2917,"William Edmondstoune Aytoun (1813–1865, S)",scotland,1813–1865
2918,"Nnorom Azuonye (born 1967, Ni)",nigeria,1967


### Strip poets names of nationality and dates

In [36]:
poets.poet = poets.poet.apply(lambda x: re.search(r'[A-z]*[ .]{0,2}[A-z]*[. ]{0,2}[A-z]*', x)[0]).copy()

In [37]:
poets

,poet,nationality,dates
0,Matthew Zapruder,united_states,1967
1,Marya Zaturenska,united_states,1902–1982
2,Robert Zend,canada,1929–1985
3,Benjamin Zephaniah,england,1958
4,Komninos Zervos,australia,1950
...,...,...,...
2915,Ayo Ayoola,nigeria,1970
2916,Pam Ayres,england,1947
2917,William Edmondstoune Aytoun,scotland,1813–1865
2918,Nnorom Azuonye,nigeria,1967


### Save poets dataframe to csv

In [38]:
poets.to_csv('poets.csv',mode = 'w', header = True, index=False)

### Use wiki scrape to find sex based on gender pronouns

In [39]:
with requests.session() as r:
    sexes = []
    male_regex = r'(?<= )[Hh][ei][ms]?(?= )'
    female_regex = r'(?<= )[SsHh][h]?[e][r]?(?= )'

    for iteration, poet in tqdm(enumerate(poets.poet)):
        if iteration % 100 == 0:
            tqdm.write(f'Scraping {poet}')
        url = f'https://en.wikipedia.org/wiki/{poet}'
        wiki = r.get(url)
        soup = BeautifulSoup(wiki.text, 'html.parser')

        try:
            intro = soup.find_all('p', class_=False)[:3]
            check = 0

            for para in intro:
                text = para.text

                if re.search(male_regex,text):
                    sexes.append('male')
                    check = 1
                    break
                elif re.search(female_regex,text):
                    sexes.append('female')
                    check = 1
                    break
            
            if check == 0:
                sexes.append(np.nan)
        except:
            tqdm.write(f'investigate {poet}')
            sexes.append(np.nan)
    
    sexes = pd.Series(sexes)
        

0it [00:00, ?it/s]

Scraping Matthew Zapruder 


100it [00:28,  3.39it/s]

Scraping Dawud Wharnsby 


200it [00:55,  2.71it/s]

...

2800it [13:30,  3.60it/s]

Scraping Toyin Adewale


2900it [14:00,  3.61it/s]

Scraping Cassandra Atherton 


2920it [14:06,  3.45it/s]


In [40]:
print(sexes.value_counts())
print(sexes.isnull().sum())

male      1577
female     842
dtype: int64
501


In [41]:
poets['sex'] = sexes

#### Drop Null Values

In [46]:
poets.dropna(how='any',subset=['sex'],inplace=True)

In [48]:
poets.to_csv('poets.csv',mode='w',header=True,index=False)

## Check poets dataframe to confirm good split of values

In [52]:
poets = pd.read_csv('poets.csv')

In [53]:
poets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2419 entries, 0 to 2418
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   poet         2419 non-null   object
 1   nationality  2411 non-null   object
 2   dates        2419 non-null   object
 3   sex          2419 non-null   object
dtypes: object(4)
memory usage: 75.7+ KB


### Convert sex to binary

1: Female

0: Male

In [55]:
poets.sex = poets.sex.apply(lambda x: 1 if x == 'female' else 0).copy()

### Convert dates column

For dates containing a range, find the midpoint and convert to float.

For poets with dates == living, convert to 2022.

Else convert date to float.

In [63]:
def convert_date(date_string):
    re_pattern = re.compile(r'(?<=\d{4})[\S]')

    if date_string == 'living':
        return float(2022)
    elif re.search(re_pattern,date_string):
        dates = re.split(re_pattern,date_string)
        return (float(dates[1]) + float(dates[0]))/2
    else:
        return float(date_string)

convert_date('1902–1982')



1942.0

In [65]:
poets.dates = poets.dates.apply(convert_date).copy()

### Clean up final null  values for nationality

In [70]:
poets[poets.nationality.isnull()]

,poet,nationality,dates,sex
105,Nissanka Wijeyeratne,NaN,1965.5,0
164,Lady Mary Wroth,NaN,1619.0,1
326,Lasana M. Sekou,NaN,1959.0,0
332,Vikram Seth,NaN,1952.0,0
487,Abhi Subedi,NaN,1945.0,0
1230,James Joyce,NaN,1911.5,0
1721,Christopher Dewdney,NaN,1951.0,0
2177,Edmund Bolton,NaN,1575.0,0


In [71]:
poets.nationality.unique()

array(['united_states', 'canada', 'england', 'australia', 'south_africa',
       'ireland', 'scotland', 'saint_lucia', 'pitcairn_islands', 'wales',
       nan, 'hong_kong', 'jamaica', 'nigeria', 'india', 'new_zealand',
       'trinidad_and_tobago', 'northern_ireland', 'ghana', 'jersey',
       'sweden', 'gambia', 'nepal', 'zimbabwe', 'newfoundland', 'guiana',
       'italy', 'barbados', 'isle_of_man', 'malta'], dtype=object)

In [72]:
poets.iloc[105,1] = 'sri_lanka'
poets.iloc[164,1] = 'england'
poets.iloc[326,1] = 'saint_martin'
poets.iloc[332,1] = 'india'
poets.iloc[487,1] = 'nepal'
poets.iloc[1230,1] = 'ireland'
poets.iloc[1721,1] = 'canada'
poets.iloc[2177,1] = 'england'

In [73]:
poets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2419 entries, 0 to 2418
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   poet         2419 non-null   object 
 1   nationality  2419 non-null   object 
 2   dates        2419 non-null   float64
 3   sex          2419 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 75.7+ KB


### Remove trailing whitespace from poet names

In [84]:
poets.poet = poets.poet.apply(lambda x: x.rstrip()).copy()

In [85]:
poets.to_csv('poets.csv',mode='w',header=True,index=False)

# PoetryFoundation

In [132]:
kp = pd.read_csv('kaggle-set/PoetryFoundationData.csv', index_col=0, header = 0, names = ['title','poem','poet','tags'])

In [133]:
kp.head()

,title,poem,poet,tags
0,\r\r\n Objects Used to Prop Open a Window\r\r\n,"\r\r\nDog bone, stapler,\r\r\ncribbage board, garlic press\r\r\n because this window is loose—lacks\r\r\nsuction, lacks grip.\r\r\nBungee cord, bootstrap,\r\r\ndog leash, leather belt\r\r\n because this window had sash cords.\r\r\nThey frayed. They broke.\r\r\nFeather duster, thatch of straw, empty\r\r\nbottle of Elmer's glue\r\r\n because this window is loud—its hinges clack\r\r\nopen, clack shut.\r\r\nStuffed bear, baby blanket,\r\r\nsingle crib newel\r\r\n because this window is split. It's dividing\r\r\nin two.\r\r\nVelvet moss, sagebrush,\r\r\nwillow branch, robin's wing\r\r\n because this window, it's pane-less. It's only\r\r\na frame of air.\r\r\n",Michelle Menting,NaN
1,\r\r\n The New Church\r\r\n,"\r\r\nThe old cupola glinted above the clouds, shone\r\r\namong fir trees, but it took him an hour\r\r\nfor the half mile all the way up the hill. As he trailed,\r\r\nthe village passed him by, greeted him,\r\r\nasked about his health, but everybody hurried\r\r\nto catch the mass, left him leaning against fences,\r\r\nmeasuring the road with the walking stick he sculpted.\r\r\nHe yearned for the day when the new church\r\r\nwould be built—right across the road. Now\r\r\nit rises above the moon: saints in frescoes\r\r\nmeet the eye, and only the rain has started to cut\r\r\nthrough the shingles on the roof of his empty\r\r\nhouse. The apple trees have taken over the sky,\r\r\nsequestered the gate, sidled over the porch.\r\r\n",Lucia Cherciu,NaN
2,\r\r\n Look for Me\r\r\n,"\r\r\nLook for me under the hood\r\r\nof that old Chevrolet settled in weeds\r\r\nat the end of the pasture.\r\r\nI'm the radiator that spent its years\r\r\nbolted in front of an engine\r\r\nshoving me forward into the wind.\r\r\nWhatever was in me in those days\r\r\nhas mostly leaked away,\r\r\nbut my cap's still screwed on tight\r\r\nand I know the names of all these\r\r\ntattered moths and broken grasshoppers\r\r\nthe rest of you've forgotten.\r\r\n",Ted Kooser,NaN
3,\r\r\n Wild Life\r\r\n,"\r\r\nBehind the silo, the Mother Rabbit\r\r\nhunches like a giant spider with strange calm:\r\r\nsix tiny babies beneath, each\r\r\nclamoring for a sweet syringe of milk.\r\r\nThis may sound cute to you, reading\r\r\nfrom your pulpit of plenty,\r\r\nbut one small one was left out of reach,\r\r\na knife of fur\r\r\nbarging between the others.\r\r\nI watched behind a turret of sand. If\r\r\nI could have cautioned the mother rabbit\r\r\nI would. If I could summon the\r\r\nBunnies to fit him in beneath\r\r\nthe belly's swell\r\r\nI would. But instead, I stood frozen, wishing\r\r\nfor some equity. This must be\r\r\nwhy it's called Wild Life because of all the\r\r\ncrazed emotions tangled up in\r\r\nthe underbrush within us.\r\r\nDid I tell you how\r\r\nthe smallest one, black and trembling,\r\r\nhopped behind the kudzu\r\r\nstill filigreed with wanting?\r\r\nShould we talk now of animal heritage, their species,\r\r\ncreature development? And what do we say\r\r\nabout form and focus—\r\r\nwriting this when a stray goes hungry, and away.\r\r\n",Grace Cavalieri,NaN
4,\r\r\n Umbrella\r\r\n,"\r\r\nWhen I push your button\r\r\nyou fly off the handle,\r\r\nold skin and bones,\r\r\nblack bat wing.\r\r\nWe're alike, you and I.\r\r\nBoth of us\r\r\nresemble my mother,\r\r\nso fierce in her advocacy\r\r\non behalf of\r\r\nthe most vulnerable child\r\r\nwho'll catch his death\r\r\nin this tempest.\r\r\nSuch a headwind!\r\r\nSometimes it requires\r\r\nall my strength\r\r\njust to end a line.\r\r\nBut when the wind is at\r\r\nmy back, we're likely\r\r\nto get carried away, and say\r\r\nsomething we can never retract,\r\r\nsomething saturated from the ribs\r\r\ndown, an old stony\r\r\nword like ruin. You're what roof\r\r\nI have, frail thing,\r\r\nyou're my argument\r\r\nagainst the whole sky.\r\r\nYou're the fundamental difference\r\r\nbetween wet and dry.\r\r\n",Connie Wanek,NaN


The PoetryFoundation database has 4 columns: `title`, `poem`, `poet`, `tags`.

Our target columns for this project are: `country_of_origin`, `sex` and `time_period`.

I need to use the poets in the PoetryFoundation dataset to identify this information.

My first step is it use the poets data scraped from wikipedia above to find the target labels for each poet.

In [134]:
kp.poet = kp.poet.apply(lambda x: x.rstrip().lstrip())

## Use Wikipedia Scraped Data

In [135]:
wiki_scrape = pd.read_csv('wiki-scrape/poets.csv')

In [136]:
wiki_scrape.head(10)

,poet,nationality,dates,sex
0,Matthew Zapruder,united_states,1967.0,0
1,Marya Zaturenska,united_states,1942.0,1
2,Robert Zend,canada,1957.0,0
3,Benjamin Zephaniah,england,1958.0,0
4,Komninos Zervos,australia,1950.0,0
5,David Zieroth,canada,1946.0,0
6,Rachel Zolf,canada,1968.0,1
7,Daniel Zomparelli,canada,2022.0,0
8,Carolyn Zonailo,canada,1947.0,1
9,Louis Zukofsky,united_states,1941.0,0


In [137]:
poems = kp.join(wiki_scrape.set_index('poet'), on = 'poet', how = 'left')

In [138]:
print(f' Value Counts:')
print(poems.sex.value_counts(normalize = True))
print()
print(f' Nulls: {poems.sex.isnull().sum()}')

 Value Counts:
0.0    0.684932
1.0    0.315068
Name: sex, dtype: float64

 Nulls: 8642


There are still 8642 rows with null values for sex. For this reason, I will use a database of baby names to label the rest of the columns.

This method is hazardous, which is why I wanted to use my wiki_scrape above first; many female poets in history use male pseudonyms to get published, so this method may mislabel some poets. Also, the dataset I am using has been compiled from statistics from the following countries:

- US
- UK
- Canada
- Australia

This means the model may not be able to label names from other countries.

### Finish labeling sex column using names db and manual effort

In [139]:
names = pd.read_csv('kaggle-set/name_gender_dataset.csv')

In [140]:
male_names = []
female_names = []

for name in names.Name.unique():
    temp = names[names.Name == name].copy()
    temp.sort_values(by = 'Gender', inplace=True)
    temp.reset_index(drop=True,inplace=True)

    if temp.shape[0] == 1:
        if temp.iloc[0,1] == 'F':
            female_names.append(name)
        else:
            male_names.append(name)
    else:
        if temp.iloc[0,2] > temp.iloc[1,2]:
            female_names.append(name)
        else:
            male_names.append(name)

In [141]:
print(len(male_names))
print(len(female_names))

50989
82921


In [142]:
poems.loc[poems['sex'].isnull(), 'sex'] = poems[poems.sex.isnull()].poet.apply(lambda x: 0.0 if x.split(' ')[0] in male_names else 1.0 if x.split(' ')[0] in female_names else np.nan).copy()

In [143]:
print(f' Value Counts:')
print(poems.sex.value_counts(normalize = True))
print()
print(f' Nulls: {poems.sex.isnull().sum()}')

 Value Counts:
0.0    0.61394
1.0    0.38606
Name: sex, dtype: float64

 Nulls: 799


In [144]:
poems.sex.isnull().sum()

799

In [145]:
sx_labels = pd.read_csv('label-sex-complete.csv')
sx_labels.head()

,poet,sex
0,Heid E. Erdrich,female
1,Indran Amirthanayagam,male
2,Osip Mandelstam,male
3,Sueyeun Juliette Lee,female
4,Sunnylyn Thibodeaux,female


In [146]:
poems.loc[poems.sex.isnull(),'sex'] = poems[poems.sex.isnull()].poet.apply(lambda x: sx_labels[sx_labels['poet'] == x].sex.values[0])

In [148]:
poems.to_csv('poems-with-targets.csv')

## Scrape wikipedia to get country and dates of remaining poets

In [113]:
remaining = poems[poems.dates.isnull()].poet.unique()

'T. S. Eliot' in remaining

True

In [96]:
soups = []
r = requests.session()
r.proxies['http'] = 'socks5h://localhost:9050'
r.proxies['https'] = 'socks5h://localhost:9050'

for index, poet in tqdm(enumerate(remaining)):
        url = f'https://en.wikipedia.org/wiki/{poet}'
        soup = BeautifulSoup(r.get(url).text, 'html.parser')
        soups.append(soup)



2524it [44:35,  1.06s/it]


In [115]:
poet_soup = pd.DataFrame(zip(remaining,soups), columns = ['poets','soup'])

In [116]:
poet_soup.head(1)

poets                                               soup
0  Michelle Menting  [html, \n, [\n, [\n, <meta charset="utf-8"/>, ...

In [121]:
poet_soup.drop(poet_soup.loc[poet_soup.poets == 'Michelle Menting'].index).head(1)

poets                                               soup
1  Lucia Cherciu  [html, \n, [\n, [\n, <meta charset="utf-8"/>, ...

#### Remove poets without wikipedia articles

In [122]:
for poet, soup in poet_soup.values:
    if soup.find('table', id = 'noarticletext'):
        poet_soup.drop(poet_soup.loc[poet_soup.poets == poet].index, inplace=True)

In [125]:
poet_soup.head(1)

poets                                               soup
2  Grace Cavalieri  [html, \n, [\n, [\n, <meta charset="utf-8"/>, ...

#### get poets info card

In [71]:
import unicodedata

def find_info_card(soup):
    try:
        infobox = soup.find('table', class_ = 'infobox vcard')
        infobox_labels = infobox.find_all('th', class_ = 'infobox-label')
        infobox_data = infobox.find_all('td', class_ = 'infobox-data')
        info_dict = {}

        for label, data in zip(infobox_labels,infobox_data):
            info_dict[unicodedata.normalize('NFKD',label.text)] = unicodedata.normalize('NFKD',data.text)

        return info_dict
    except:
        return np.nan
    

In [44]:
poet_soup['infocard'] = poet_soup.soup.apply(find_info_card)


In [246]:
poet_soup['dates'] = poet_soup.infocard.apply(find_dates_from_card)

In [115]:
def find_country_of_origin(card):
    try:
        if 'Nationality' in card.keys():
            return card['Nationality']
        elif 'Place of birth' in card.keys():
            return card['Place of birth']
        elif 'Born' in card.keys():
            if re.search(r'[A-z ,]*', card['Born']):
               return re.findall(r'([A-z ,]*)', card['Born'])
        else:
            return np.nan
    except:
        return np.nan
        

In [116]:
poet_soup.country_of_origin = poet_soup.infocard.apply(find_country_of_origin).copy()

In [117]:
poet_soup[['poets','country_of_origin','dates','infocard']]

,poets,country_of_origin,dates,infocard
0,Grace Cavalieri,American,[],"{'Nationality': 'American', 'Alma mater': 'Goddard University, New Jersey State Teachers College at Trenton, University of Maryland, College Park', 'Genre': 'Poetry, Playwright, Radio'}"
1,Simone White,NaN,None,NaN
2,Heid E. Erdrich,"Turtle Mountain Ojibwe, American",1963.0,"{'Born': ' (1963-11-26) November 26, 1963 (age 58)', 'Occupation': 'Writer', 'Nationality': 'Turtle Mountain Ojibwe, American', 'Education': 'Dartmouth College, BA', 'Alma mater': 'Johns Hopkins University, master's degrees', 'Relatives': 'Louise Erdrich (sister)'}"
3,Sarah White,NaN,None,NaN
4,Judith Harris,NaN,None,NaN
...,...,...,...,...
1544,Lisa Barnett,NaN,None,NaN
1545,Jeanne Murray Walker,"[ , , , , , , , , , , , , , May , , , , , , , , , , , age , , , , Parkers Prairie, Minnesota, ]",1944.0,"{'Born': ' (1944-05-27) May 27, 1944 (age 77)Parkers Prairie, Minnesota', 'Occupation': 'Writer', 'Alma mater': 'Wheaton CollegeLoyola UniversityThe University of Pennsylvania', 'Genre': 'Poetry', 'Spouse': 'E. Daniel Larkin III'}"
1546,Dylan Thomas,"[Dylan Marlais Thomas, , , , , , , , , , , , , , , October , , , , , Swansea, Wales, United Kingdom, ]",1933.5,"{'Born': 'Dylan Marlais Thomas(1914-10-27)27 October 1914Swansea, Wales, United Kingdom', 'Died': '9 November 1953(1953-11-09) (aged 39)Greenwich Village, New York City, United States', 'Resting place': 'Laugharne, Carmarthenshire, Wales', 'Occupation': 'Poet and writer', 'Spouse': 'Caitlin Macnamara ​(m. 1937)​', 'Children': '3, including Aeronwy Bryn Thomas'}"
1547,John Wheelwright,NaN,None,NaN


In [118]:
print(poet_soup.country_of_origin.isnull().sum())
print(poet_soup.dates.isnull().sum())
print(poet_soup.shape)

967
1001
(1549, 6)


In [119]:
def find_dates_from_card(card):
    try:
        dates = []
        if 'Born' in card.keys():
            dates.append(float(re.search(r'\d{4}',card['Born'])[0]))
        elif 'Date of birth' in card.keys():
           dates.append(float(re.search(r'\d{4}',card['Date of birth'])[0]))
        
        
        if 'Died' in card.keys():
            dates.append(float(re.search(r'\d{4}',card['Died'])[0]))
        elif 'Date of death' in card.keys():
            dates.append(float(re.search(r'\d{4}',card['Date of death'])[0]))
        
        if len(dates) == 1:
            return dates[0]
        elif len(dates) == 2:
            return sum(dates)/len(dates)
        else:
            return dates
    except:
        pass

In [60]:
poet_soup = pd.read_csv('poet-soup.csv')

In [62]:
poet_soup.soup = poet_soup.soup.apply(lambda x: BeautifulSoup(x, 'html.parser'))

In [72]:
poet_soup['infocard'] = poet_soup.soup.apply(find_info_card)


In [73]:
poet_soup.infocard

0       {'Nationality': 'American', 'Alma mater': 'God...
1                                                     NaN
2       {'Born': ' (1963-11-26) November 26, 1963 (age...
3                                                     NaN
4                                                     NaN
                              ...                        
1544                                                  NaN
1545    {'Born': ' (1944-05-27) May 27, 1944 (age 77)P...
1546    {'Born': 'Dylan Marlais Thomas(1914-10-27)27 O...
1547                                                  NaN
1548                                                  NaN
Name: infocard, Length: 1549, dtype: object

In [80]:
poet_soup.dates = poet_soup.infocard.apply(find_dates_from_card)

In [82]:
poet_soup[['poets','infocard','dates','country_of_origin']][~poet_soup.infocard.isnull()]

,poets,infocard,dates,country_of_origin
0,Grace Cavalieri,"{'Nationality': 'American', 'Alma mater': 'Goddard University, New Jersey State Teachers College at Trenton, University of Maryland, College Park', 'Genre': 'Poetry, Playwright, Radio'}",[],American
2,Heid E. Erdrich,"{'Born': ' (1963-11-26) November 26, 1963 (age 58)', 'Occupation': 'Writer', 'Nationality': 'Turtle Mountain Ojibwe, American', 'Education': 'Dartmouth College, BA', 'Alma mater': 'Johns Hopkins University, master's degrees', 'Relatives': 'Louise Erdrich (sister)'}",1963.0,"Turtle Mountain Ojibwe, American"
5,Truong Tran,"{'Born': 'Truong Tran1969Saigon, Vietnam', 'Occupation': 'Poet, Visual artist', 'Nationality': 'Vietnamese American', 'Alma mater': 'UC Santa Cruz', 'Notable works': 'Placing the Accents (1999) dust and consciousness (2002)', 'Notable awards': 'San Francisco Poetry Center Book Prize (2002)Western States Book Prize Finalist in Poetry (1999)'}",1969.0,Vietnamese American
9,Eugene Gloria,"{'Alma mater': 'San Francisco State UniversityMiami UniversityUniversity of Oregon', 'Genre': 'poetry'}",[],NaN
10,Indran Amirthanayagam,"{'Born': '1960Colombo', 'Genre': 'Poetry', 'Notable awards': 'Paterson Poetry Prize'}",1960.0,"['', '', '', '', 'Colombo', '']"
...,...,...,...,...
1532,Robert Marteau,"{'Born': 'February 8th, 1925Virollet, France', 'Died': 'May 16th, 2011Paris, France'}",1968.0,"['February ', '', 'th, ', '', '', '', '', 'Virollet, France', '']"
1538,Deirdre O'Connor,"{'Preceded by': 'Barry Maddern', 'Succeeded by': 'Geoffrey Giudice', 'Born': 'Deirdre Frances O'Connor (1941-02-05) 5 February 1941 (age 80)Sydney, New South Wales, Australia', 'Nationality': 'Australian', 'Spouse(s)': 'Gregory Sachs, Michael Joseph', 'Children': 'David, Antony, Peter, Lionel & Sean', 'Alma mater': 'Bethlehem College, AshfieldUniversity of Sydney', 'Occupation': 'Judge, academic, lawyer'}",1941.0,Australian
1540,Al Maginnes,"{'Position:': 'Tackle', 'Born:': '(1897-04-05)April 5, 1897Boston, Massachusetts, United States', 'Died:': 'January 30, 1966(1966-01-30) (aged 68)', 'Height:': '6 ft 1 in (1.85 m)', 'Weight:': '188 lb (85 kg)', 'High school:': 'Boston (MA) English', 'College:': 'Lehigh'}",[],NaN
1545,Jeanne Murray Walker,"{'Born': ' (1944-05-27) May 27, 1944 (age 77)Parkers Prairie, Minnesota', 'Occupation': 'Writer', 'Alma mater': 'Wheaton CollegeLoyola UniversityThe University of Pennsylvania', 'Genre': 'Poetry', 'Spouse': 'E. Daniel Larkin III'}",1944.0,"[' ', '', '', '', '', '', '', '', '', '', '', '', '', ' May ', '', '', ', ', '', '', '', '', ' ', '', 'age ', '', '', '', 'Parkers Prairie, Minnesota', '']"


In [120]:
has_infocard = poet_soup[['poets','infocard','dates','country_of_origin']][~poet_soup.infocard.isnull()].copy()

In [121]:
has_infocard.reset_index(drop=True, inplace=True)

In [127]:
has_infocard.country_of_origin = has_infocard.country_of_origin.apply(lambda x: [word.strip() for word in x if word.strip() != ''] if type(x) == list else x).copy()

In [128]:
has_infocard.head()

,poets,infocard,dates,country_of_origin
0,Grace Cavalieri,"{'Nationality': 'American', 'Alma mater': 'Goddard University, New Jersey State Teachers College at Trenton, University of Maryland, College Park', 'Genre': 'Poetry, Playwright, Radio'}",[],American
1,Heid E. Erdrich,"{'Born': ' (1963-11-26) November 26, 1963 (age 58)', 'Occupation': 'Writer', 'Nationality': 'Turtle Mountain Ojibwe, American', 'Education': 'Dartmouth College, BA', 'Alma mater': 'Johns Hopkins University, master's degrees', 'Relatives': 'Louise Erdrich (sister)'}",1963.0,"Turtle Mountain Ojibwe, American"
2,Truong Tran,"{'Born': 'Truong Tran1969Saigon, Vietnam', 'Occupation': 'Poet, Visual artist', 'Nationality': 'Vietnamese American', 'Alma mater': 'UC Santa Cruz', 'Notable works': 'Placing the Accents (1999) dust and consciousness (2002)', 'Notable awards': 'San Francisco Poetry Center Book Prize (2002)Western States Book Prize Finalist in Poetry (1999)'}",1969.0,Vietnamese American
3,Eugene Gloria,"{'Alma mater': 'San Francisco State UniversityMiami UniversityUniversity of Oregon', 'Genre': 'poetry'}",[],NaN
4,Indran Amirthanayagam,"{'Born': '1960Colombo', 'Genre': 'Poetry', 'Notable awards': 'Paterson Poetry Prize'}",1960.0,[Colombo]


In [149]:
poems.nationality.unique()


array([nan, 'united_states', 'england', 'trinidad_and_tobago',
       'hong_kong', 'canada', 'ireland', 'scotland', 'australia',
       'northern_ireland', 'wales', 'india', 'ghana', 'saint_lucia',
       'jamaica', 'new_zealand', 'barbados'], dtype=object)

In [156]:
has_infocard[['poets','country_of_origin']].to_csv('label_countries.csv')

In [157]:
labelled_countries = pd.read_csv('datasets/helper_sets/label_countries_final.csv')

In [161]:
poems.sex.isnull().sum()

0

In [169]:
poems.nationality.isnull().sum()

8642

In [172]:
poems.loc[poems.nationality.isnull(), 'nationality'] = poems.loc[poems.nationality.isnull()]['poet'].apply(lambda x: labelled_countries[labelled_countries.poets == x].country_of_origin.values[0] if x in labelled_countries.poets.unique() else np.nan).copy()

In [177]:
poems.loc[poems.dates.isnull(),'dates'] = poems.loc[poems.dates.isnull()]['poet'].apply(lambda x: has_infocard[has_infocard.poets == x].dates.values[0] if x in has_infocard.poets.unique() else np.nan).copy()

In [178]:
print(f'Missing nationalities: {poems.nationality.isnull().sum()}')
print(f'Missing sex: {poems.sex.isnull().sum()}')
print(f'Missing dates: {poems.dates.isnull().sum()}')

Missing nationalities: 5798
Missing sex: 0
Missing dates: 6059


In [193]:
poems.reset_index(drop=True,inplace=True)

In [206]:
poems['number_of_lines'] = poems.poem.apply(lambda x: len([word for word in x.splitlines() if word != ''])).copy()

In [208]:
poems.drop(poems.loc[poems['number_of_lines'] <= 1], axis = 1)

""
0
1
2
3
4
...
13893
13894
13895
13896


In [217]:
poems.drop(poems.loc[poems['number_of_lines'] == 0].index, inplace=True)

In [222]:
poems.dates = poems.dates.apply(lambda x: np.nan if x == [] else x).copy()

In [230]:
poems.drop_duplicates(inplace=True)


,title,poem,poet,tags,nationality,dates,sex,number_of_lines
6,\r\r\n Invisible Fish\r\r\n,"\r\r\nInvisible fish swim this ghost ocean now described by waves of sand, by water-worn rock. Soon the fish will learn to walk. Then humans will come ashore and paint dreams on the dying stone. Then later, much later, the ocean floor will be punctuated by Chevy trucks, carrying the dreamers’ decendants, who are going to the store.\r\r\n",Joy Harjo,"Living,Time & Brevity,Relationships,Family & Ancestors,Nature,Landscapes & Pastorals,Seas, Rivers, & Streams,Social Commentaries,History & Politics",united_states,1951.0,1.0,1
7,\r\r\n Don’t Bother the Earth Spirit\r\r\n,"\r\r\nDon’t bother the earth spirit who lives here. She is working on a story. It is the oldest story in the world and it is delicate, changing. If she sees you watching she will invite you in for coffee, give you warm bread, and you will be obligated to stay and listen. But this is no ordinary story. You will have to endure earthquakes, lightning, the deaths of all those you love, the most blinding beauty. It’s a story so compelling you may never want to leave; this is how she traps you. See that stone finger over there? That is the only one who ever escaped.\r\r\n",Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fairy-tales & Legends",united_states,1951.0,1.0,1
9,"\r\r\n [""Hour in which I consider hydrangea""]\r\r\n","\r\r\nHour in which I consider hydrangea, a salt or sand plant, varietal, the question of varietals, the diet of every mother I know, 5 pounds feels like 20, I have lost … I have lost, yes, a sense of my own possible beauty, grown external, I externalize beauty. Beauty occurs on the surface of plants; the sun darkens the skin of my child, he is so small, he is beautiful (I can see; it is obvious) and everything about him is beautiful. His hand swells from the bite [spread?] of some insect[’s] venom because he is small. He appears to feel nothing. He smashes his skull against the floor. He screams. I hold him in my lap on the kitchen floor in front of an open freezer, pressing a pack of frozen clay against his forehead. He likes the cold. I see; it is so obvious. Hydrangea. When I move, when I walk pushing my child’s stroller (it is both walking and pushing or hauling, sometimes, also, lifting; it is having another body, an adjunct body composed of errand and weight and tenderness and no small amount of power), I imagine I can feel this small amount of weight, this 5 pounds like 20, interfering with the twitch of every muscle in my body. As an object, a mother is confusing, a middle-aged mother with little spare flesh, I feel every inch of major muscle pulling against gravity and against the weight of my child, now sleeping. This is the hour for thinking hydrangea. Let no man look at me. I stop to brush the drowsy child’s little eye. His face. He barely considers his mother. I am all around him. Why should he consider what is all around him? Perhaps what is missing is a subtle power of differentiation. I am in, therefore, a time of mass apprehensions.\r\r\n",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Trees & Flowers",NaN,NaN,1.0,1
80,"\r\r\nfrom What the Heart Longs For When It Only Knows Heat [""We spend the afternoon together watching a docudrama...""]\r\r\n","\r\r\nWe spend the afternoon together watching a docudrama about wild horses that roamed the ancient Arctic Circle. Surprisingly sleek, built for speed and not the weather, they were remarkable for their recklessness. They careen headlong down ice bluffs to fall into a broken heap. We can hear the small, tinny sounds of their terror as they plunge across vast, glowing glacial faces. All of this takes place alongside an abstractly relentless gunmetal sea. I can feel you turn to me, wetness marking the corners of your lips and eyes. I, too, am mesmerized, my vision limited to a sense of motion on the peripheries. Later, I am summoned for an impromptu scan and, miraculously, I pass.\

In [247]:
poems['number_of_stanzas'] = poems.poem.apply(lambda x: len([stanza for stanza in re.split(r'\r\r\n \r\r\n', x) if stanza != ''])).copy()

In [251]:
poems.head()

,title,poem,poet,tags,nationality,dates,sex,number_of_lines,number_of_stanzas
0,\r\r\n Objects Used to Prop Open a Window\r\r\n,"\r\r\nDog bone, stapler,\r\r\ncribbage board, garlic press\r\r\n because this window is loose—lacks\r\r\nsuction, lacks grip.\r\r\nBungee cord, bootstrap,\r\r\ndog leash, leather belt\r\r\n because this window had sash cords.\r\r\nThey frayed. They broke.\r\r\nFeather duster, thatch of straw, empty\r\r\nbottle of Elmer's glue\r\r\n because this window is loud—its hinges clack\r\r\nopen, clack shut.\r\r\nStuffed bear, baby blanket,\r\r\nsingle crib newel\r\r\n because this window is split. It's dividing\r\r\nin two.\r\r\nVelvet moss, sagebrush,\r\r\nwillow branch, robin's wing\r\r\n because this window, it's pane-less. It's only\r\r\na frame of air.\r\r\n",Michelle Menting,NaN,NaN,NaN,1.0,20,1
1,\r\r\n The New Church\r\r\n,"\r\r\nThe old cupola glinted above the clouds, shone\r\r\namong fir trees, but it took him an hour\r\r\nfor the half mile all the way up the hill. As he trailed,\r\r\nthe village passed him by, greeted him,\r\r\nasked about his health, but everybody hurried\r\r\nto catch the mass, left him leaning against fences,\r\r\nmeasuring the road with the walking stick he sculpted.\r\r\nHe yearned for the day when the new church\r\r\nwould be built—right across the road. Now\r\r\nit rises above the moon: saints in frescoes\r\r\nmeet the eye, and only the rain has started to cut\r\r\nthrough the shingles on the roof of his empty\r\r\nhouse. The apple trees have taken over the sky,\r\r\nsequestered the gate, sidled over the porch.\r\r\n",Lucia Cherciu,NaN,NaN,NaN,1.0,14,1
2,\r\r\n Look for Me\r\r\n,"\r\r\nLook for me under the hood\r\r\nof that old Chevrolet settled in weeds\r\r\nat the end of the pasture.\r\r\nI'm the radiator that spent its years\r\r\nbolted in front of an engine\r\r\nshoving me forward into the wind.\r\r\nWhatever was in me in those days\r\r\nhas mostly leaked away,\r\r\nbut my cap's still screwed on tight\r\r\nand I know the names of all these\r\r\ntattered moths and broken grasshoppers\r\r\nthe rest of you've forgotten.\r\r\n",Ted Kooser,NaN,united_states,1939.0,0.0,12,1
3,\r\r\n Wild Life\r\r\n,"\r\r\nBehind the silo, the Mother Rabbit\r\r\nhunches like a giant spider with strange calm:\r\r\nsix tiny babies beneath, each\r\r\nclamoring for a sweet syringe of milk.\r\r\nThis may sound cute to you, reading\r\r\nfrom your pulpit of plenty,\r\r\nbut one small one was left out of reach,\r\r\na knife of fur\r\r\nbarging between the others.\r\r\nI watched behind a turret of sand. If\r\r\nI could have cautioned the mother rabbit\r\r\nI would. If I could summon the\r\r\nBunnies to fit him in beneath\r\r\nthe belly's swell\r\r\nI would. But instead, I stood frozen, wishing\r\r\nfor some equity. This must be\r\r\nwhy it's called Wild Life because of all the\r\r\ncrazed emotions tangled up in\r\r\nthe underbrush within us.\r\r\nDid I tell you how\r\r\nthe smallest one, black and trembling,\r\r\nhopped behind the kudzu\r\r\nstill filigreed with wanting?\r\r\nShould we talk now of animal heritage, their species,\r\r\ncreature development? And what do we say\r\r\nabout form and focus—\r\r\nwriting this when a stray goes hungry, and away.\r\r\n",Grace Cavalieri,NaN,united_states,NaN,1.0,27,1
4,\r\r\n Umbrella\r\r\n,"\r\r\nWhen I push your button\r\r\nyou fly off the handle,\r\r\nold skin and bones,\r\r\nblack bat wing.\r\r\nWe're alike, you and I.\r\r\nBoth of us\r\r\nresemble my mother,\r\r\nso fierce in her advocacy\r\r\non behalf of\r\r\nthe most vulnerable child\r\r\nwho'll catch his death\r\r\nin this tempest.\r\r\nSuch a headwind!\r\r\nSometimes it requires\r\r\nall my strength\r\r\njust to end a line.\r\r\nBut when the wind is at\r\r\nmy back, we're likely\r\r\nto get carried away, and say\r\r\nsomething we can never retract,\r\r\nsomething saturated from the ribs\r\r\ndown, an old stony\r\r\nword like ruin. You're what roof\r\r\nI have, frail thing,\r\r\nyou're my argument\r\r\n

In [260]:
poems.to_csv(f'datasets/poems-with-targets-{now.strftime("%d-%m-%Y")}.csv',mode='w')

In [253]:
now = datetime.now()

In [254]:
now

datetime.datetime(2022, 2, 2, 9, 35, 21, 98670)

In [257]:
print(now.strftime('%d/%m/%Y'))

02/02/2022
